In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import torch
import torch.nn.functional as F

from transformers import AutoModelForCausalLM, AutoTokenizer

from synth import CrossEntropyDifferential

In [16]:
device = torch.device("cuda")

model_name = "microsoft/Phi-3.5-mini-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

cross_entropy_differential = CrossEntropyDifferential(model, tokenizer, device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.63it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 108.00 MiB. GPU 0 has a total capacity of 31.72 GiB of which 107.88 MiB is free. Process 151397 has 1.29 GiB memory in use. Including non-PyTorch memory, this process has 29.39 GiB memory in use. Process 153611 has 848.00 MiB memory in use. Of the allocated memory 28.88 GiB is allocated by PyTorch, and 143.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 

In [ ]:
model

Phi3Model(
  (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
  (embed_dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-31): 32 x Phi3DecoderLayer(
      (self_attn): Phi3Attention(
        (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
        (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
      )
      (mlp): Phi3MLP(
        (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
        (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
        (activation_fn): SiLU()
      )
      (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      (resid_attn_dropout): Dropout(p=0.0, inplace=False)
      (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
    )
  )
  (norm): Phi3RMSNorm((3072,), eps=1e-05)
)

In [14]:
Y = """ ::START::thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Angelo told me that his favourite pizza is the quattroformaggi rossa with salsiccia. 
When I was a kid, I used to watch the pizza maker create his pizzas, he was from Romania but a very nice gentleman I have to say."""

prompt = f"""<|system|>
Write a small phrase that would fit well at the ::START:: the phrase<|end|>
<|user|>
{Y}<|end|>
<|assistant|>"""
input = tokenizer(prompt, return_tensors='pt', padding=True).to(device)

output = model.generate(
    input.input_ids,
    num_return_sequences=20,
    max_new_tokens=15,
    do_sample=True,
    temperature=1
)

TypeError: The current model class (Phi3Model) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'Phi3ForCausalLM'}

In [5]:
with torch.no_grad():
    goutput = model(**input)

goutput[0].shape, input.input_ids.shape

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


(torch.Size([1, 117, 32064]), torch.Size([1, 117]))

In [6]:
output[0][input.input_ids.shape[1]:].shape

torch.Size([15])

In [7]:
print(tokenizer.decode(output[4]))

<|system|> Write a small phrase that would fit well at the ::START:: the phrase<|end|><|user|> ::START::thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Angelo told me that his favourite pizza is the quattroformaggi rossa with salsiccia. 
When I was a kid, I used to watch the pizza maker create his pizzas, he was from Romania but a very nice gentleman I have to say.<|end|><|assistant|> "Delighting in the irresistible blend of rich m


In [8]:
results = []
for i in output:
    answ = tokenizer.decode(i[len(input.input_ids[0]):], skip_special_tokens=True)
    results.append((
        cross_entropy_differential(answ, Y, diff=True).item(),
        answ
        ))
best_result = min(results, key=lambda x: x[0])
best_result

ValueError: Expected input batch_size (96) to match target batch_size (97).

In [ ]:
Y = """ thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Angelo told me that his favourite pizza is the quattroformaggi rossa with salsiccia. 
When I was a kid, I used to watch the pizza maker create his pizzas, he was from Romania but a very nice gentleman I have to say."""

prompt = f"""{Y}\n::INTRODUCE::"""
input = tokenizer(prompt, return_tensors='pt', padding=True).to(device)

outputs = torch.tensor([])
for i in range(3):
    torch.cuda.empty_cache()
    output = model.generate(
        input.input_ids,
        attention_mask=input.attention_mask,
        max_length=len(input.input_ids[0])+6,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=1
    )
    outputs = torch.cat((outputs, output.to(torch.device('cpu'))), dim=0)

results = []
for i in output:
    answ = tokenizer.decode(i[len(input.input_ids[0]):], skip_special_tokens=True)
    results.append((
        cross_entropy_differential(answ, Y, diff=True).item(),
        answ
        ))

best_result = min(results, key=lambda x: x[0])

AttributeError: 'CausalLMOutputWithPast' object has no attribute 'input_ids'

In [ ]:
Y = """ thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Angelo told me that his favourite pizza is the quattroformaggi rossa with salsiccia. 
When I was a kid, I used to watch the pizza maker create his pizzas, he was from Romania but a very nice gentleman I have to say."""

prompt = f"""{Y}\n::INTRODUCE::"""
input = tokenizer(prompt, return_tensors='pt', padding=True).to(device)

outputs = torch.tensor([])
for i in range(3):
    torch.cuda.empty_cache()
    output = model.generate(
        input.input_ids,
        attention_mask=input.attention_mask,
        max_length=len(input.input_ids[0])+6,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=1
    )
    outputs = torch.cat((outputs, output.to(torch.device('cpu'))), dim=0)

results = []
for i in output:
    answ = tokenizer.decode(i[len(input.input_ids[0]):], skip_special_tokens=True)
    results.append((
        cross_entropy_differential(answ, Y, diff=True).item(),
        answ
        ))
best_result = min(results, key=lambda x: x[0])

You are not running the flash-attention implementation, expect numerical differences.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


ValueError: Expected input batch_size (93) to match target batch_size (94).

In [ ]:
print(best_result)

(-0.20408201217651367, ' \xa0My favourite Italian slice')
